[spherical_geometry](https://github.com/spacetelescope/spherical_geometry/releases/tag/1.2.20)

In [7]:
import spherical_geometry.polygon
import scipy
import zipfile
import os
import pandas as pd
import json
import numpy as np
import scipy.spatial
import astropy.coordinates
import shapely.geometry.polygon, shapely.geos
import logging
import tqdm,tqdm.notebook

def astropy_q_to_deg(x):
    return float(np.array(x.to("deg")))

TODO:
- [x] download data
- [x] calculate spherical voronoi
- [x] load into `spherical_geometry`
- [x] find Japan's geojson
- [x] compute intersections
- [ ] visualize intersections
- [ ] compute radii
- [ ] compute res
- [ ] write text
- [ ] convert to markdown
- [ ] make pull req
- [ ] oversee acceptance
- [ ] publish

In [2]:
# assume file `S12-20_GML.zip` from https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-S12-v2_7.html
# was downloaded into `Downloads` folder under the name `S12-20_GML.zip`
# change the next line appropriately if this is not so
zf = zipfile.ZipFile(f"{os.environ['HOME']}/Downloads/S12-20_GML.zip")
assert "S12-20_GML/S12-20_NumberOfPassengers.geojson" in zf.namelist(), "make sure you have the right archive"
zf.extract("S12-20_GML/S12-20_NumberOfPassengers.geojson","/tmp/1224bf86_54a4_4e74_963f_7ebdbc7bac70")

with open('/tmp/1224bf86_54a4_4e74_963f_7ebdbc7bac70/S12-20_GML/S12-20_NumberOfPassengers.geojson', errors="ignore") as f:
    data = json.load(f)
data = pd.DataFrame(data["features"])
# cf. https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-S12-v2_7.html
data = pd.DataFrame({
    "name": data.properties.apply(lambda d: d["S12_001"]),
    "n_passengers": data.properties.apply(lambda d: d["S12_041"]),
    "lonlat": data.geometry.apply(lambda g: tuple(sum(map(np.array, g["coordinates"]))/len(g["coordinates"]))),
})
data = pd.DataFrame([slice_.iloc[0] for _,slice_ in data.groupby("lonlat")])
print(len(data))
data.head()

,name,n_passengers,lonlat
523,那覇空港,14008,"(127.652285, 26.20666)"
528,赤嶺,4841,"(127.660665, 26.193265)"
527,小禄,7707,"(127.66703000000001, 26.196550000000002)"
529,奥武山公園,4438,"(127.67524, 26.200654999999998)"
521,旭橋,8749,"(127.67548, 26.21195)"


In [3]:
points = np.array(astropy.coordinates.spherical_to_cartesian(1,np.radians(data.lonlat.apply(lambda t:t[1])),np.radians(data.lonlat.apply(lambda t:t[0]))))
points = points.transpose()

In [4]:
#Spherical Voronoi threshold (taken from https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.SphericalVoronoi.html)
DEFAULT_SPHERICAL_VORONOI_THRESHOLD = 1e-06
duplicates = scipy.spatial.cKDTree(points).query_pairs(DEFAULT_SPHERICAL_VORONOI_THRESHOLD)
assert {a for a,b in duplicates} & {b for a,b in duplicates} == set()
filtered_points = np.array([pt for i,pt in enumerate(points) if i not in {b for a,b in duplicates}])

In [5]:
sv = scipy.spatial.SphericalVoronoi(
    filtered_points,
    center=np.zeros(3),
    radius=1,
    threshold=DEFAULT_SPHERICAL_VORONOI_THRESHOLD,
)

In [8]:
polygons = []
sg_polygons = []
for pt,region in tqdm.notebook.tqdm(list(zip(filtered_points,sv.regions))):
    n = len(region)
    region_points = [sv.vertices[region][i] for i in range(n)]
    region_points = region_points+region_points[0]
    pts = []
    for rp in region_points:
        pts.append([float(np.array(astropy.coordinates.cartesian_to_spherical(*rp)[i].to("degree"))) for i in range(2,0,-1)])
    polygons.append(shapely.geometry.polygon.Polygon(pts))
    
    sg_polygons.append(spherical_geometry.polygon.SingleSphericalPolygon(region_points, pt))
polygons;

In [12]:
pd.Series([polygon.contains_lonlat(*point) for point,polygon in zip(sv.points,sg_polygons)]).value_counts()
# sg_polygons[0].contains_lonlat(*sv.points[0])

True     8655
False    1234
dtype: int64

In [11]:
# I assume you have downloaded the file `countries.geojson` from https://datahub.io/core/geo-countries 
# to `Downloads/countries.geojson` in your home folder
with open(f"{os.environ['HOME']}/Downloads/countries.geojson") as f:
    countries_geojson = json.load(f)

japan_geojson = [f["geometry"] for f in countries_geojson["features"] if f["properties"]["ADMIN"]=="Japan"][0]

polygons = japan_geojson["coordinates"]
biggest_polygon = max(polygons,key=lambda polygon:shapely.geometry.polygon.Polygon(polygon[0]).area)
biggest_polygon;

#taken from https://www.latlong.net/place/tokyo-japan-8040.html
tokyo_latlon = 35.652832, 139.839478

_df = pd.DataFrame(biggest_polygon[0],columns=["lon","lat"])
japan_sg_polygon = spherical_geometry.polygon.SingleSphericalPolygon(
    #points
    np.array(astropy.coordinates.spherical_to_cartesian(1,np.radians(_df.lat),np.radians(_df.lon))).transpose(),
    #inside
    inside=astropy.coordinates.spherical_to_cartesian(1,*np.radians(tokyo_latlon)),
)
japan_polygon = shapely.geometry.polygon.Polygon(biggest_polygon[0])

In [13]:
%%time
japan_sg_polygon.intersection(sg_polygons[0])

CPU times: user 6min 42s, sys: 2.16 s, total: 6min 44s
Wall time: 6min 48s


[SingleSphericalPolygon(array([[-0.58699322,  0.46792191,  0.66067242],
       [-0.58711885,  0.46815556,  0.66039521],
       [-0.58720845,  0.46836321,  0.66016827],
       ...,
       [-0.58689955,  0.46755277,  0.66101689],
       [-0.58690197,  0.4677587 ,  0.66086904],
       [-0.58699322,  0.46792191,  0.66067242]]), array([ 0.5808175 , -0.54698242, -0.60287748]))]

In [26]:
from datetime import timedelta
(timedelta(minutes=6,seconds=48)*len(sg_polygons)).total_seconds()/60**2/24

46.698055555555555

In [192]:
import geopy.distance
import seaborn as sns

dists = []
for s,e in zip(biggest_polygon[0][:-1],biggest_polygon[0][1:]):
    d = geopy.distance.distance(s[::-1],e[::-1]).meters
    dists.append(d)
pd.Series(dists).describe()

count     2478.000000
mean      2403.367099
std       2257.181420
min         46.832563
25%       1232.016900
50%       1829.557480
75%       2778.677174
max      31696.133952
dtype: float64

In [376]:
_s = []
for i,(pt,polygon) in enumerate(zip(sv.points,polygons)):
    _,lat,lon = astropy.coordinates.cartesian_to_spherical(*pt)
    _pt = shapely.geometry.Point(astropy_q_to_deg(lon),astropy_q_to_deg(lat))
    _s.append(polygon.contains(_pt))
pd.Series(_s).value_counts()

False    7955
True     1934
dtype: int64

In [307]:
intersections = []
_error_count = 0
for i,polygon in tqdm.notebook.tqdm(list(enumerate(polygons))):
    try:
        intersection = japan_polygon.intersection(polygon)
        intersections.append((i,intersection))
    except shapely.geos.TopologicalError as te:
        logging.warning(te)
        _error_count += 1
# intersections = [japan_polygon.intersection(polygon) for polygon in tqdm.notebook.tqdm(polygons)]
_error_count

ERROR:shapely.geos:TopologyException: Input geom 1 is invalid: Self-intersection at or near point 277.0375295488376 38.075180445995954 at 277.0375295488376 38.075180445995954


1

In [312]:
pd.Series([intersection[1].geom_type for intersection in intersections]).value_counts()

Polygon         9871
MultiPolygon      17
dtype: int64

In [42]:
%%time
pd.Series([
    pd.Series([japan_sg_polygon.contains_point(pt) for pt in sg_polygon._points]).all()
    for sg_polygon
    in tqdm.notebook.tqdm(sg_polygons)
]).value_counts()


CPU times: user 21min 53s, sys: 8.65 s, total: 22min 1s
Wall time: 22min 12s


True     6750
False    3139
dtype: int64

In [93]:
voronoi_edges = set()

for sg_polygon,region in tqdm.notebook.tqdm(list(zip(sg_polygons,sv.regions))):
#     n = len(region)
#     region_points = [sv.vertices[region][i] for i in range(n)]
#     region_points = region_points+region_points[0]
#     region_points = list(map(tuple,region_points))
#     if not pd.Series([japan_sg_polygon.contains_point(pt) for pt in sg_polygon._points]).all():
    region_points = sg_polygon.points
    for s,e in zip(region_points[:-1],region_points[1:]):
        if japan_sg_polygon.contains_point(s)!=japan_sg_polygon.contains_point(e):
            voronoi_edges.add(tuple(list(s)+list(e)))
len(voronoi_edges)

1425

In [104]:
import itertools
intersection_vertices = set()
for e1,e2 in tqdm.notebook.tqdm(list(itertools.product(zip(japan_sg_polygon.points[:-1],japan_sg_polygon.points[1:]), voronoi_edges))):
    e2 = list(e2)
    iv = spherical_geometry.great_circle_arc.intersection(*e1,e2[:3],e2[3:])
    if not np.isnan(iv).any():
        intersection_vertices.add(tuple(iv))
#     if len(intersection_vertices)>5:
#         break

In [109]:
inner_polygons = [
    sg_polygon
    for sg_polygon
    in tqdm.notebook.tqdm(sg_polygons)
    if pd.Series([japan_sg_polygon.contains_point(p) for p in sg_polygon.points]).all()
]
len(inner_polygons),len(sg_polygons)

(6750, 9889)

In [124]:
pd.Series(inner_polygons).apply(lambda p:p.is_clockwise()).value_counts()

True     3771
False    2979
dtype: int64

In [130]:
pd.DataFrame(inner_polygons[0].to_lonlat())

,0,1,2,3,4,5,6
0,131.050067,130.937112,131.059526,131.059977,131.036295,131.059606,131.050067
1,34.180206,34.189016,34.200912,34.202548,34.208087,34.205106,34.180206


In [126]:
_df = pd.DataFrame([
    {
        "geojson": json.dumps({
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [list(zip(*p.to_lonlat())) if not p.is_clockwise() else list(zip(*p.to_lonlat()))[::-1]]
            }
        })
    }
    for p
    in inner_polygons
])
_df.to_csv("/tmp/8b4bdfa2_255a_4a03_a0c1_63aad0faa1c0.csv",index=None)
_df

,geojson
0,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
2,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
4,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
...,...
6745,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
6746,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
6747,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
6748,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."


In [342]:
_df = pd.DataFrame([
    {
        "cnt": i,
        "geojson": json.dumps({
            "type": "Feature",
            "geometry": {
                "type": p.geom_type,
                "coordinates": [
                    [
                        [lon,lat]
                        for lon,lat
                        in zip(*pp.exterior.coords.xy)
                    ]
                    for pp
                    in ([p] if p.geom_type=="Polygon" else p.geoms)
                    if len(pp.exterior.coords)>0
                ]
            }
        }),
        **{
            k:v
            for k,v
            in zip(["lat","lon"],map(lambda x:float(np.array(x.to("degree"))),astropy.coordinates.cartesian_to_spherical(*sv.points[i])[1:]))
        }
    }
    for i, p
    in intersections
    if sum(map(lambda p:len(p.exterior.coords),[p] if p.geom_type=="Polygon" else p.geoms))>0
])
_df.to_csv("/tmp/e623adc7_61c4_409d_ad8d_25939e5d94f6.csv", index=None)
# _df.geojson.iloc[0]
_df

,cnt,geojson,lat,lon
0,836,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",34.070063,130.903637
1,838,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",34.128040,130.912000
2,842,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",34.047270,130.915755
3,843,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",34.024892,130.916725
4,847,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",33.949150,130.921607
...,...,...,...,...
7330,9652,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",39.619299,141.946367
7331,9653,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",39.639895,141.947365
7332,9654,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",39.463590,141.951300
7333,9656,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...",39.447235,141.959440
